# PART1

## Json import to openSearch

In [ ]:
import createRecipesMap as crm

crm.CreateRecipesMap("../jsonData/recipe_queries_results.json", '../jsonData/recipesMap.json').createMap()

## Login to openSearch

In [ ]:
import loginOpenSearch as lop

login = lop.LoginOpenSearch("../jsonData/openSearchConfig.json").login()
    
client = login[0]
index_name = login[1]

## Create index structure

In [ ]:
import createIndexes as ci

ci.CreateIndexes(client, index_name).createIndexStructure()

## Choose the tokenizer and model that will be used in the embeddings

In [ ]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/msmarco-distilbert-base-v2")
model = AutoModel.from_pretrained("sentence-transformers/msmarco-distilbert-base-v2")

In [ ]:
import computeEmbeddings as ce

ce.ComputeEmbeddings("../jsonData/recipesMap.json","../jsonData/recipesEmbeddingsMap.json", tokenizer, model).createMap()

In [ ]:
import indexRecipes as ir

ir.IndexRecipes("../jsonData/recipesMap.json", "../jsonData/recipesEmbeddingsMap.json", client, index_name).indexRecipes()


In [ ]:
import search as s

s.Search(client, index_name, tokenizer, model).queryOpenSearch('Holiday Salad', 5,None, None, ["salads"], ["lupine"], None)

## Testing

In [ ]:
import searchJson as sj
metrics = sj.SearchJson(client, index_name, tokenizer, model)
metrics.searchJson("../jsonData/cleanAnnotations.json")

In [ ]:
delete this sentence to run
if client.indices.exists(index=index_name):
    # Delete the index.
    response = client.indices.delete(
        index = index_name,
        timeout = "600s"
    )
    print('\nDeleting index:')
    print(response)

# PART2

## Compute the step similarity matrix

### Compute the steps json files

In [ ]:
import stepsCalculator as sc

sc.CreateStepsMaps("../jsonData/recipesMap.json", '../jsonData/stepsImage.json', '../jsonData/stepsNoImage.json').createMaps()


### Compute the embeddings

In [ ]:
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"

device

In [ ]:
import computeImagesEmbeddings as cie

cie.ComputeImagesEmbeddings("../jsonData/stepsImage.json","../jsonData/stepsNoImage.json","../jsonData/stepsImageEmbeddingsMap.json","../jsonData/stepsNoImageEmbeddingsMap.json","images/").createMap()

### Compute the similarity matrix

In [ ]:
import computeMatrix as cm

imageSimilarityMatrix = cm.ComputeMatrix("../jsonData/stepsImageEmbeddingsMap.json", "../jsonData/stepsNoImageEmbeddingsMap.json").createMatrix()
print(imageSimilarityMatrix)
print(len(imageSimilarityMatrix[0]))

In [ ]:
import getImage as gi

gi.GetImage(imageSimilarityMatrix, "../jsonData/stepsImage.json", "../jsonData/stepsNoImage.json", "images/","../jsonData/recipesMap.json").getImageFunc()

In [ ]:
#import computeClipOutput as cco

#image = cco.ComputeClipOutput(imageSimilarityMatrix, "../jsonData/stepsImage.json", "../jsonData/stepsNoImage.json", "images/","../jsonData/recipesMap.json").getImage()

# PART3


In [ ]:
import os
import numpy as np
import transformers
import json



import random

from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding
)

model_finetuned = '/user/data/public/twiz-intent-model'
with open(os.path.join(model_finetuned + '/all_intents.json'), 'r') as all_intents_json:
    all_intents = json.load(all_intents_json) # contains the written out names of intents. also implicitly

tokenizer_name = 'roberta-base' # try 'bert-base-uncased', 'bert-base-cased', 'bert-large-uncased'
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name) # loads a tokenizer

model = AutoModelForSequenceClassification.from_pretrained(model_finetuned, num_labels=len(all_intents)) # Loads the BERT model weights



In [ ]:
import stateMachine
turing = stateMachine.StateMachine(tokenizer,model, all_intents)
print('aye wat recipe u want')
intent = turing.getIntent('hello')
getattr(turing, intent)()
intent = turing.getIntent('go back')
getattr(turing, intent)()
intent = turing.getIntent('cookie')
getattr(turing, intent)()
intent = turing.getIntent('chocolate')
getattr(turing, intent)()


In [ ]:
import stateMachine
turing = stateMachine.StateMachine(tokenizer,model, all_intents)
while(turing.state != 'terminar'):

    sentence = 'hello'
    intent = turing.getIntent('hello')
    try: 
        ## getattr(turing, intent, 'AMAZON.StopIntent')()
        getattr(turing, 'AMAZON.StopIntent')() 
    except Exception as e:
        print ('pls repeat')
        continue

        